In [1]:
# jupyter notebook with an example for an interferometer calculation

import sympy as sy
from sympy import I

from Interferometry import Interferometer, Hamiltonian, Pulse, U

# define symbolic variables
m1,m2,g1,g2,k,T,m,g=sy.symbols("m1,m2,g1,g2,k,T,m,g")






Define Hamiltonians by passing a parameter list [a,b,c,d,e]
 corresponding to the Hamiltonian
 H=a*p^2 + b*p +c*(px+xp) + d*x + e
 
 

In [2]:
H=Hamiltonian([1/(2*m), 0,0,m*g,0])

print('Hamiltonian is:')         
print(H)


Hamiltonian is:


\hat{p}**2/(2*m) + \hat{x}*g*m

Define upper and lower sequence (note the ordering of operators is the same as if acting on a initial ket state)

In [3]:

upperSequence=[U(H,T),  Pulse(-k), U(H,T), Pulse(k)] 
lowerSequence=[Pulse(-k),  U(H,T),  Pulse(k), U(H,T)]

print(Pulse(-k))


implements:


exp(-I*\hat{x}*k)

initialize an interferometer object with the upper and lower sequence

In [6]:

I=Interferometer(upperSequence, lowerSequence)

#calculate phase
_,res=I.phase()

res


-I*T**2*g*k